# Autocomplete

In this project we will explore different ways to create an autocomplete algorithm: similar to the ones you might find on a phone when you type. Given a partial sentence we will try to predict the word that comes next. It turns out this seemingly simple task is an incredibly important problem in modern natural language processing (NLP), and much effort has gone in to solving it.

The way we'll be solving this problem is by using something called a _**bigram model**_. Don't worry about what this is for now; just know that we will have to do the following steps:

- Download the data (in this project, we'll be using "The Complete Works of Shakespeare")
- Clean the data (the text we get is messy, so we have to tidy it up a bit)
- Train our model
- Predict words and sentences

# Setting Up

Before we begin we should import some Python packages that will be useful later on. Run the following block of code to import these libraries, and take a look at the comments to see what each import does:

In [ ]:
import numpy as np                      # Vector and matrix math library (makes math a lot easier)
import random                           # Functions for random operations
import re                               # Regex (makes cleaning text a lot easier)
from collections import Counter         # Counter utility 
from tqdm import tqdm                   # Progress bar (makes life nicer)
import matplotlib.pyplot as plt         # Library to plot and visualize data
from collections import defaultdict     # A special dictionary implementation
import json                             # Library to parse JSON files

# The Data

As with all machine learning algorithms, before we begin we first need to find data to train with. In this project we will start off by using the "Complete Works of Shakespeare" which, as its name suggests, contains the totality of Shakespeare's writings. 

In NLP people call a dataset of text a _**corpus**_. So we will call our dataset the "Shakespeare corpus."

Run the following block to download the corpus and load it into python. If at any time you want to completely restart, just run this block of code again:

In [ ]:
# Download the shakespeare corpus to `shakespeare.txt`
!wget -O shakespeare.txt https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt
#!sed -i '40000,$d' shakespeare.txt

# Read the dataset into python
with open('./shakespeare.txt', 'r+') as f:
  lines = f.readlines()

# Exploring and Cleaning the Data

Data from the internet is rarely neat and tidy. Oftentimes there will be extraneous information in our data that we don't want. Other times the data will be formatted oddly, so we'll want to standardize everything. In this section we'll take a look at our data and then _**clean**_ the corpus accordingly.

First, try exploring the dataset by answering the following questions (the first one is done for you):

(You can find the full dataset [here](https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt) in a form that is a bit more readable)

In [ ]:
# The dataset is in the variable `lines`. It is a list of strings, one
# for each line in the dataset. What does the first line say?
print(lines[0])



# How many lines are in the dataset?

### YOUR CODE HERE ###

print(### YOUR ANSWER HERE ###



# How many characters are in the dataset?

### YOUR CODE HERE ###

print(### YOUR ANSWER HERE ###



# What do the first 10 lines of the corpus say?

### YOUR CODE HERE ###

print(### YOUR ANSWER HERE ###

Taking a look at your answer to the last question, it should be obvious that the first couple lines of the dataset don't actually contain any Shakespeare! In fact, the beginning and end of this dataset is filled with copyright information, which we don't want. 

We would much rather have a dataset of _only_ Shakespeare's writings, so we should _clean_ these pieces of text:

In [ ]:
# At which line does the actual Shakespeare start? (hint: it's somewhere around line 250)

### YOUR CODE HERE ###

print(### YOUR ANSWER HERE ###



# The last couple lines is also copyright text
# At which line does the Shakespeare end? 
# (hint: it's somewhere around line 124440, or -20)

### YOUR CODE HERE ##

print(### YOUR ANSWER HERE ###

In [ ]:
# Write some code to remove the copyright info 
# from the beginning and end of the dataset
lines = ### YOUR CODE HERE ##

Actually, there's more copyright text sprinkled all over the dataset. We would like to remove these lines as well. Below we've given a list of lines that we don't want in the `blacklist` variable. Write some code to remove any lines from our dataset that matches a line in the blacklist:

In [ ]:
blacklist = ['<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM\n',
             'SHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS\n',
             'PROVIDED BY PROJECT GUTENBERG ETEXT OF ILLINOIS BENEDICTINE COLLEGE\n',
             'WITH PERMISSION.  ELECTRONIC AND MACHINE READABLE COPIES MAY BE\n',
             'DISTRIBUTED SO LONG AS SUCH COPIES (1) ARE FOR YOUR OR OTHERS\n',
             'PERSONAL USE ONLY, AND (2) ARE NOT DISTRIBUTED OR USED\n',
             'COMMERCIALLY.  PROHIBITED COMMERCIAL DISTRIBUTION INCLUDES BY ANY\n',
             'SERVICE THAT CHARGES FOR DOWNLOAD TIME OR FOR MEMBERSHIP.>>\n']


# Remove lines from `lines` if they appear in the black list

### YOUR CODE HERE ### 



# Test to make sure you're on track (don't touch this line!)
assert len(lines) == 122455, "The new `lines` list should have 122455 lines in it!"

Great, we've now removed most of the copyright text that we don't care about! 

Another problem is that our corpus is just a list of strings right now. What we really want is a list of _sentences_. Try doing this by answering the questions in the blocks below:

In [ ]:
# Turn the corpus from a list of strings into one big string
# Hint: the `join` function might be helpful (but not necessary) (try googling it!)
text = ### YOUR CODE HERE ###



# Test to make sure you're on track (don't touch this line!)
assert len(text) == 5461111, "`text` should have 5461111 characters in it if processed correctly!"

In [ ]:
# Lowercase the string (hint: try googling "lowercase string python")
text = ### YOUR CODE HERE ###



# Test to make sure you're on track (don't touch this line!)
assert text.islower(), "`text` is not completely lower case!"

Now we want to remove anything that is not a letter or a punctuation mark (one of '.', '!', or '?'), any additional whitespace, and then split the string into sentences. We've written this code for you, so just read the next block and run run it.

(The code below uses the python library `re`, which takes advantage of _**regular expressions**_. Regular expressions give a really powerful way to search through text. You can learn more about them [here](https://regexone.com/).)

In [ ]:
# Remove all non-alphabetical letters and non-punctuation
pattern = re.compile('[^a-zA-Z_ .!?]+')
text = pattern.sub('', text)

# Remove extra whitespace
text = re.sub("\s+"," ", text)

# Get sentences by splitting the string on any
# punctuation mark ('.', '!' or '?')
sentences = re.split('[.!?]', text)

At this point we've done everything we need to clean the data! The sentences should be in a variable called `sentences` now. Of course, the data is still a bit messy, but nobody's perfect! 

You should explore the sentences we now have by playing around with the block below. And then try to answer the questions below in the block afterwards:

In [ ]:
# Finds "to be or not to be" in the corpus, and outputs the containing sentence
# Feel free to play with this block of code!

for idx, sentence in enumerate(sentences):
  if 'to be or not to be' in sentence:
    target_idx = idx
print(sentences[target_idx])
print('Index: ' + str(target_idx))

In [ ]:
# Can you find a line from "Romeo and Juliet"?

### YOUR CODE HERE ###

print(### YOUR ANSWER HERE ###



# How many sentences do we have total?

### YOUR CODE HERE ###

print(### YOUR ANSWER HERE ###



# What is the longest sentence and how long is it?
# Can you understand what the sentence is and why it's so long?

### YOUR CODE HERE ###

print(### YOUR ANSWER HERE ###

# The Bigram Model

The Bigram Model sounds fancy, but it's actually pretty straightforward. Say we want to know what word should come after the word `"computer"`. We just look at our dataset, find all instances of the word `"computer"`, look at the word that comes _after_ `"computer"`, and tally up those words. Then we randomly choose a word based on how frequently it appears in our tally.

For example, let's say we've looked at our dataset and the following words appear after `"computer"` (with the frequency in parentheses):

- `"software"` (4)
- `"chip"` (2)
- `"virus"` (3)
- `"desk"` (1)
- `"bananas"` (0)

Then given the question "what word comes after `"computer"`?" We would answer with the word `"software"` with a 40% probability, the word `"chip"` with a 20% probability, and so on. We should answer with the word `"bananas"` 0% of the time, because the word `"bananas"` _never_ comes after the word `"computer"` in our dataset.

So in order to train this model, we need to build a dictionary that when given a word, gives us the tallies of the next word in our dataset.

## What _is_ a Bigram???

A _**bigram**_ is just a fancy way term for "two words in a row". For example, the sentence:

`"to be or not to be that is the question"`

contains the bigrams `"to be"`, `"be or"`, `"or not"`, `"not to"`, `"to be"`, `"be that"`, `"that is"`, `"is the"`, and `"the question"`. In a bigram model, we're essentially tallying up the number of bigrams in our dataset, and using these tallies to predict the next word.

Google provides a service that lets you plot out the frequency of various bigrams and n-grams (`n` words in a row) in books over time. You can try it out [here](https://books.google.com/ngrams/graph?content=artificial+intelligence&year_start=1800&year_end=2019&corpus=26&smoothing=3&direct_url=t1%3B%2Cartificial%20intelligence%3B%2Cc0).

## Word Counts

To start, let's count all the words in our dataset first. To do this we will use the Python string function `split` and the `Counter` utility:

In [ ]:
# Split the corpus into a list of consecutive words (hint: Googling `split` function is useful here)

### YOUR CODE HERE ###

words = ### YOUR CODE HERE ###



# Use the Counter class to count the words in the corpus
# (hint: the Python documentation could be helpful here: "https://docs.python.org/3/library/collections.html#collections.Counter")

word_freq = ### YOUR CODE HERE ###




# Test to make sure you're on track (don't touch thes lines!)
assert len(words) == 882602, "`words` must be a list with 882602 elements!"
assert type(word_freq) is Counter, "`word_freq` must be a `Counter` object!"

If everything was done correctly, then `word_freq` should be a `Counter` object. Let's explore the data using this object. Answer the following questions:

In [ ]:
# What are the 10 most common words used by Shakespeare and how often are they used?
# (hint: The Counter's `most_common` function is useful)

### YOUR CODE HERE ###

print(### YOUR ANSWER HERE ###



# How many unique words are there in Shakespeare's complete works?

### YOUR CODE HERE ###

print(### YOUR ANSWER HERE ###



# How many times does the word "wherefore" appear?

### YOUR CODE HERE ###

print(### YOUR ANSWER HERE ###



# How many times does the word "romeo" appear?

### YOUR CODE HERE ###

print(### YOUR ANSWER HERE ###

So we lied to you. We won't actually be training our model for _every single word_ in the dataset. There are almost 30,000 unique words in the corpus, and finding the tallies for each word would take just too long, even for a computer! We're only going to look at the 1,000 most common words, which actually still gives pretty decent results:

In [ ]:
# Make a list containing the 1000 most common words in our corpus
# We will use this list later
# (hint: `most_common` will help here)

### YOUR CODE HERE ### 

top_1000_words = ### YOUR ANSWER HERE ###



# Test to make sure you're on track (don't touch this line!)
assert 'whether' in top_1000_words and 'wert' in top_1000_words, "`top_1000_words` seems to be incorrect!"

# Finding Bigrams

Now let's find all the bigrams in our corpus (regardless if they're the most common words or not). Iterate through the list `sentences` and create a new list of strings, where each string is a bigram:

In [ ]:
# Create a list of bigrams by looping through the list `sentences`
# The format of the list should be:
# ['to be', 'be or', 'or not', 'not to', ...]

bigrams = []
for sentence in sentences:

  # Make a list of words in the sentence
  words_in_sentence = ### YOUR CODE HERE ###

  # Loop through each word in the sentence (except the last)
  for idx in range(len(words_in_sentence) - 1):

    # Create the bigram from this word and the next
    bigram = ### YOUR CODE HERE ###

    bigrams.append(bigram)

# you can print out the first couple entries to make sure you
# have a sensible answer by running the following:
print(bigrams[:10])



# Test to make sure you're on track (don't touch this line!)
assert 'motion seeming' in bigrams and 'forebetrayed and' in bigrams, "You seem to be missing some bigrams!"

# Training our Model

Now let's make a dictionary that tallies the "next word" given a word. This will be the real workhorse of our algorithm.

In detail, we will make a dictionary of dictionaries called `bigram_freqs` so that when we query `bigram_freqs[first_word][second_word]` we will get the number of times the `"second_word"` comes after `"first_word"` in our dataset.

Again, we can't do this for every word in the dataset, so we'll only look at the 1000 most common words:

Now fill out the blanks in the code below to generate our bigram frequency counter from `bigrams`, our list of bigrams. The code should take about 10 seconds to run.

In [ ]:
# Create bigram freqs matrix (don't worry about this code too much,
# or feel free to ask about this)
tally_factory = lambda: dict([(w, 1/1000) for w in top_1000_words])
bigram_tallies = defaultdict(tally_factory)
    
# Fill up our bigram dictionary
for bigram in tqdm(bigrams):    # tqdm gives us a progress bar

    # Get the first and second word of our bigram
    first_word, second_word = ### YOUR CODE HERE ###

    # Check to make sure `first_word` and `second_word` are in the
    # top 1000 most common words
    if ### YOUR CODE HERE ###:

        # Update the tallies
        bigram_tallies[first_word][second_word] += ### YOUR CODE HERE ###



# Test to make sure you're on track (don't touch this line!)
assert bigram_tallies['where']['art'] == 10.001, "`bigram_tallies` seems to be incorrect!"

Finally, let's write a function that predicts the next word given a word. Instead of choosing the next word exactly in proportion to the "next word" tallies, we'll do something slightly easier: Given a word, we will randomly choose one of the 20 most common "next words" to go next. Fill out the blanks in the next block to do this:

In [ ]:
def predict_next_word(word):
    '''
    Given `word`, find the top 10 most common words that
    come next, and randomly return one of them
    '''
    # Only allow predictions on most common words
    assert word in top_1000_words, "Not a common word!"
    
    # Get list of tallies for the word
    # (hint: dictionaries have a function called `.values()`)
    tallies = ### YOUR CODE HERE

    # Get the index of the 20 most common words
    # (this is done for you)
    top_20_idxs = np.array(list(tallies)).argsort()[-20:]

    # Get an index
    # (hint: you need to select a random 
    # entry from the list `top_20_idxs`)
    # (hint: try Googling)
    rand_idx = ### YOUR CODE HERE ###
    
    # Find and return word the `rand_idx`th
    # word from top_1000_words
    next_word = ### YOUR CODE HERE ###
    return next_word

Let's try out our function a few times on the word `"his"` a few times by re-running the next code block. Also try out other words by changing the argument of the function! For example, what words tend to come after `"her"`?

You may notice that the words following `"his"` sometimes tend to be more masculine like `"sword"` or `"majesty"` while the words following `"her"` tend to be more feminine such as `"love"` or `"husband"`. This is because our prediction function is trained on Shakespeare's works, and as a result reflects his style of writing. Broadly, the idea that the data you train on affects your downstream model's predictions is called _**Dataset Bias**_. We will talk more about this later on.

In [ ]:
# Return a prediction of the next word
print(predict_next_word('her'))

Sometimes on a phone keyboard it's fun to just let autocomplete write entire sentences and see where it goes. We can do someting similar with our text prediction model. Let's start with the word `"my"` and predict the next word using our function, then feed that word _back into the next-word predictor_. Doing this a few times we can generate a sentence. 

Concretely, if we start with the word `"when"` and our algorithm predicts the word `"is"`, we should feed the word `"is"` _back into_ our predictor. Say we then get the word `"he"`. We now have the sentence `"when is he..."`. Continuing this process we can predict any number of words into the future.

In computer science, we call this an _**autoregressive model**_ because it regresses to it's own output.

Try to create a simple autoregressive model below using the function `predict_next_word` and run it a few times to see if it generates any sensible sentences:

In [ ]:
def autoregress(word):
  ### YOUR CODE HERE ###

autoregress('when')

# Dataset Bias

Your autoregressive model should come up with some fairly decent sentences, and also some sentences that don't make any sense (this is a pretty simple model after all!). However, one thing you may notice is that all the sentences are very "Shakespearian." This shouldn't be too big a surprise, because we trained our bigram model on Shakespeare! The formal term for this effect is called _**Dataset Bias**_, and we already saw an example of this earlier, when we looked at what sorts of words tended to follow the word `"his"` vs. `"her"`. Dataset bias is when the dataset we choose has an effect on our model's predictions.

Let's try training a bigram model on a different dataset to see dataset bias in action. Run the following code to download, load, and clean a dataset of Amazon reviews on computer software products (this can take a bit of time, because the dataset is pretty large):

In [ ]:
# Download amazon reviews from the interwebz and unzip it
!wget -O amazon_software_reviews.json.gz http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Software.json.gz
!gzip --force -d amazon_software_reviews.json.gz

# Load the data into python
with open('amazon_software_reviews.json', 'r+') as f:
    reviews = f.readlines()
    
# Extract all the text and clean
reviews = [json.loads(review).get('reviewText', '') for review in reviews]
reviews = reviews[:int(len(reviews) * .1)]  # Only use 10% of dataset
amazon_text = ' '.join(reviews)
amazon_text = amazon_text.lower()
pattern = re.compile('[^a-zA-Z_ .!?]+')
amazon_text = pattern.sub('', amazon_text)
amazon_text = re.sub("\s+"," ", amazon_text)
sentences = re.split('[.!?]', amazon_text)

Now all of the Amazon review sentences are in the variable `sentences` (we've overwritten our Shakespeare sentences). This means you can go back to the section called **The Bigram Model** and rerun all the blocks below to train a bigram model on these Amazon reviews. Go ahead and do that, and see what sorts of sentences you get.

You should have gotten sentences that look something like

- `"when youre doing some very fast pc running a better off"`
- `"when we do some serious computer i am extremely happy and"`
- `"when using windows in my new interface in that the computer"`

these sentences are very different from the Shakespeare sentences, and this is because we're using a different dataset. 

Sometimes dataset bias can manifest itself in pretty obvious ways, like what we've just seen. But other times, dataset bias can be problematic. If the dataset we're using contains bad language, or racism, or sexism, then our model might learn these tendencies. For this reason, we have to be really careful when choosing and cleaning our data. For more information, check [this article](https://www.vox.com/videos/2021/3/31/22348722/ai-bias-racial-machine-learning) out.

# Modern Language Models

The bigram model that we've been creating was first developed in the 1950's by Claude Shannon (a University of Michigan alumni!). Because these are such old and basic models, you might have noticed that a lot of the time the sentences just didn't make any sense. 

NLP has come a long way since the 50's. Nowadays, the best language models are class of models called _**transformers**_. To give you an idea of how powerful these models are, let's play around with some!

We will use a model called [GPT-2](https://openai.com/blog/better-language-models/), released by OpenAI, a company that specializes in AI research. GPT-2 is one of the best models today, and is trained on data taken from Wikipedia, Reddit, and other places on the internet. Run the code below to download all the necessary components (this may take around a minute):

In [ ]:
!pip install transformers

# import relevant packages
from transformers import pipeline

# create a pipline and specify the gpt2 model for text generation
generator = pipeline('text-generation', model='gpt2')

After running the above block, we should have downloaded GPT-2 and set it up for text prediction. To play around with it, check out the next block of code. 

Interesting things to think about include:

- Is the model simply memorizing sentences from the internet? One way you can test this is Googling the output of the model to see if people have written it before.
- Can you get the model to mess up with crazy inputs? If so what inputs did you use?
- Try changing the `max_length` parameter to output longer predictions. Does the output of the model make any sense? Is it consistent over long predictions?
- Can the model answer questions? Try giving the model questions (such as `"What color is the sky?"`) as input.
- Does the model tend to have any biases? Does it tend to talk about a particular topic excessively?

In [ ]:
# Given the partial sentence, GPT-2 will try to continue it
# Try changing input sentence and rerunning this cell to see what GPT-2 outputs
partial_sentence = "When in the course of human events"
generator(partial_sentence, max_length=30, num_return_sequences=1)

# The End

Hopefully now you have some understanding of what NLP is! Just to recap, we've:

- Explored large datasets of text
- Learned how to clean these large datasets of text
- Built an autocomplete function
- Investigated the effects of data bias 
- Played around with modern language models

If you're still curious you can try to complete another notebook, or ask the instructors for more things to do. Other things to try in this notebook include:

- Trying out other text datasets 
- Cleaning the Shakespeare data better
- Try to find other sources of bias in the Shakespeare dataset